In [1]:
import os.path
import numpy as np
import copy

arquivo = os.path.join('projeto.csv') 

dataRDD = (sc.textFile(arquivo, 8))

In [2]:
lpRDD = dataRDD.map(lambda l: l.split(';')).map(lambda l: {'label':float(l[0]),'features':[float(l[1]),float(l[2]),l[3]]})

In [3]:
def contador(lp, feats, splits):
    r = []
    for feat in feats:
        splitss = splits[feat]
        for split in splitss:
            if feat==3:
                r.append( ((feat,split), ((0,lp['label']), (0,lp['label']**2), (0,1))))
            elif feat==0 or feat==1:
                if lp['features'][feat] >= split:
                    r.append( ((feat,split), ((0,lp['label']), (0,lp['label']**2), (0,1))))
                else:
                    r.append( ((feat,split), ((lp['label'],0), (lp['label']**2,0), (1,0))))
                
            elif feat==2:                
                if lp['features'][feat] in split:
                    r.append( ((feat,split), ((0,lp['label']), (0,lp['label']**2), (0,1))))
                else:
                    r.append( ((feat,split), ((lp['label'],0), (lp['label']**2,0), (1,0))))
                
    return r

In [4]:
def soma(t1,t2):
    return  ((t1[0][0]+t2[0][0],t1[0][1]+t2[0][1]), #((0.668333333, 0), (0.44666944399888886, 0), (1, 0))
             (t1[1][0]+t2[1][0],t1[1][1]+t2[1][1]),
             (t1[2][0]+t2[2][0],t1[2][1]+t2[2][1]))

In [5]:
def calc_stats(t):
    if t[1][2][0] > 0:
        mmenor = t[1][0][0] / t[1][2][0]
        sdmenor = (t[1][1][0] - (t[1][0][0]**2 / t[1][2][0]))/(t[1][2][0])
        cvmenor = np.sqrt(sdmenor)/mmenor*100
        menor = (sdmenor, cvmenor,t[1][2][0])
    else:
        menor = (0, 0, 0)
    
    if t[1][2][1] > 0:    
        mmaior = t[1][0][1] / t[1][2][1]
        sdmaior = (t[1][1][1] - (t[1][0][1]**2 / t[1][2][1]))/(t[1][2][1])
        cvmaior = np.sqrt(sdmaior)/mmaior*100
        maior = (sdmaior, cvmaior,t[1][2][1])
    else:
        maior = (0, 0, 0)
    return (t[0], menor, maior)

In [6]:
def sdreduction(t,c):
    #return ((t[0], c[2][0] - t[1][0]*(t[1][2]/c[2][2]) - t[2][0]*(t[2][2]/c[2][2]))) 
    return ((t[0], c[2][0] - (t[1][2]/c[2][2])*t[1][0] - (t[2][2]/c[2][2])*t[2][0], (t[1][1], t[2][1]))) 

In [7]:
def prepara(t):
    
    return ((t['features'][0],t['features'][0]),(t['features'][1],t['features'][1]), set((t['features'][2],)))
    
def minmax(t1,t2):
    return ((min(t1[0][0],t2[0][0]),max(t1[0][1],t2[0][1])),
            (min(t1[1][0],t2[1][0]),max(t1[1][1],t2[1][1])),
             t1[2].union(t2[2]))

In [49]:
import time
def tree(lpRDD, maxDepth=1, currDepth=0, st=None,verbose=1, arvore={}, pos=0):
    if st!=None:
        if st[0]<10 or maxDepth<currDepth or st[1]<0:
            media = lpRDD.map(lambda x: x['label']).mean()
            arvore[pos] = media
            print (st, media, currDepth, pos, lpRDD.count())
            return arvore #[caminho, (direction, media)]    
    
    t0 = time.time()
    if verbose >= 2:
        print ('total: %s'%lpRDD.count())

    (min0,max0),(min1,max1),periods =lpRDD.map(prepara).reduce(minmax)
    
    if verbose >= 2:
        print ( "min-max-uniques:", (min0,max0),(min1,max1),periods)
    if verbose >= 2:
        print ('calc minmax: %.2f s'%(time.time()-t0))
    
    periods = list(periods)

    
    distsplits = [i*(max0-min0)/5. for i in range(1,5)]
    timesplits = [i*(max1-min1)/5. for i in range(1,5)]    
    perisplits = [tuple([periods[j] for j in range(i+1)]) for i in range(len(periods)-1)]

    feats = [0,1,2,3]
    splits = [distsplits,timesplits,perisplits,['full']] 
    
    results = lpRDD.flatMap(lambda lp: contador(lp,feats,splits)).reduceByKey(soma)
    
    if verbose >= 2:
        print ('contador: %.2f s'%(time.time()-t0))
    statistics = results.map(lambda c: calc_stats(c)).filter(lambda x: x[0][0]<3)
    if verbose >= 2:
        print ('estatísticas: %.2f s'%(time.time()-t0))
    
    statistics3 = results.map(lambda c: calc_stats(c)).filter(lambda x: x[0][0]==3)
    classe = statistics3.collect()[0]
    es = statistics.map(lambda x: sdreduction(x,classe)).collect()  
    e = max(es, key = lambda t: t[1]) #atributo / split escolhido
    
    if verbose >= 1:
        print ('atributo escolhido: %.2f s'%(time.time()-t0))
        print (e)
        
        
    if e[0][0]<2:
            menorRDD = lpRDD.filter(lambda v: v['features'][e[0][0]]<e[0][1])
            if verbose >= 2:
                print ('filtro menor01: %.2f s'%(time.time()-t0))
            maiorRDD = lpRDD.filter(lambda v: v['features'][e[0][0]]>e[0][1])   
            if verbose >= 2:
                print ('filtro maior01: %.2f s'%(time.time()-t0))
    elif e[0][0]==2:
            menorRDD = lpRDD.filter(lambda v: v['features'][e[0][0]] in e[0][1])
            if verbose >= 2:
                print ('filtro menor2: %.2f s'%(time.time()-t0))
            maiorRDD = lpRDD.filter(lambda v: v['features'][e[0][0]] not in e[0][1])
            if verbose >= 2:
                print ('filtro maior2: %.2f s'%(time.time()-t0))
    
    #caminho.append((direction,e[0]))
    #camenor = copy.deepcopy(caminho)
    #camaior = copy.deepcopy(caminho)
    arvore[pos] = e[0]
    print (e[0], pos)
    
    if menorRDD.count()>1:
        amenor = tree(menorRDD, maxDepth, currDepth+1,(e[2][0], e[1]),verbose,arvore,pos=(2*pos+1))
  
    if maiorRDD.count()>1:
        amaior = tree(maiorRDD, maxDepth, currDepth+1,(e[2][1], e[1]),verbose,arvore,pos=(2*pos+2))
    return arvore

In [ ]:
ti=time.time()
arvore = tree(lpRDD,maxDepth=16,verbose=0)
tf = time.time()

(1, 2.658277778) 0
(1, 1.5948333336) 1
(1, 1.2757777775999999) 3
(1, 1.0204444448) 7


In [ ]:
print('Tempo total de construção: %.2f'%(tf-ti))

In [ ]:
arvore